In [1]:
# !unzip proofs.zip -d proofs

In [2]:
# Step 1: Install elan (Lean toolchain manager)
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y

# Step 2: Update Python process PATH so subprocess.run() can find `lean`
import os
elan_bin_path = os.path.expanduser("~/.elan/bin")
os.environ["PATH"] = elan_bin_path + ":" + os.environ["PATH"]

# Verify the installation by checking the version
!lean --version

import os
import subprocess

def setup_lean_project(project_dir="/tmp/lean_project"):
    """
    Creates a Lean project, configures it to use Mathlib,
    and downloads pre-compiled library files.
    """
    print(f"--- Setting up Lean project in: {project_dir} ---")
    os.makedirs(project_dir, exist_ok=True)

    # Content for the lakefile.lean
    lakefile_content = """
    import Lake
    open Lake DSL

    package «lean_project»

    require mathlib from git
      "https://github.com/leanprover-community/mathlib4.git"

    @[default_target]
    lean_lib «lean_project»
    """
    # Write the lakefile
    with open(os.path.join(project_dir, "lakefile.lean"), "w") as f:
        f.write(lakefile_content)

    # Run `lake exe cache get` to download Mathlib's pre-compiled files
    # This is much faster than building from source.
    print("--- Downloading Mathlib cache (this may take a few minutes)... ---")
    try:
        subprocess.run(
            ["lake", "exe", "cache", "get"],
            cwd=project_dir,
            check=True,
            capture_output=True,
            text=True
        )
        print("--- Mathlib cache downloaded successfully. ---")
    except subprocess.CalledProcessError as e:
        print("❌ Error setting up Mathlib cache.")
        print(f"--- STDOUT ---\n{e.stdout}")
        print(f"--- STDERR ---\n{e.stderr}")
        raise  # Stop execution if setup fails

    return project_dir

# --- Call this function once at the start of your script ---
lean_project_path = setup_lean_project()
lean_project_path

info: downloading installer
info: default toolchain set to 'stable'
Lean (version 4.24.0, x86_64-unknown-linux-gnu, commit 797c613eb9b6d4ec95db23e3e00af9ac6657f24b, Release)
--- Setting up Lean project in: /tmp/lean_project ---
--- Downloading Mathlib cache (this may take a few minutes)... ---
--- Mathlib cache downloaded successfully. ---


'/tmp/lean_project'

In [3]:
import utils
print(utils.get_proof_variants)

<function get_proof_variants at 0x719a1f8c2320>


In [4]:
import inspect
print(inspect.getsource(utils.get_proof_variants))

def get_proof_variants(s: str) -> List[str]:
    return [s] + apply_bulk_strategies(s)



In [26]:
import subprocess
import os
import re
# from concurrent.futures import ProcessPoolExecutor, as_completed
from utils import get_proof_variants
from typing import Dict
import threading
import concurrent.futures
import tempfile

LOG_PATH = os.path.expanduser("~/error.log")   # expand ~ -> /home/you/...
os.makedirs(os.path.dirname(LOG_PATH) or ".", exist_ok=True)
_log_lock = threading.Lock()

def check_lean_proof(proof_and_context: Dict, log_errors=False, max_workers=None) -> bool:
    """
    Checks a Lean‑4 proof string inside the given project using `lake`.
    If any variant succeeds, the *first* successful proof is saved to:
        corrected_proofs/<problem_id>/<proof_solver>/<attempt_id>.txt
    Returns True if a proof was saved, otherwise False.
    """
    # Verify the top‑level keys that must be present
    assert "proof" in proof_and_context, \
        "Missing 'proof' key – you need a proof string to test."
    assert "formal_statement" in proof_and_context, \
        "Missing 'formal_statement' key – you have to give the theorem statement."
    assert "project_dir" in proof_and_context, \
        "Missing 'project_dir' key – cannot locate the Lean project."
    assert "metadata" in proof_and_context, \
        "Missing 'metadata' key – you'll need context such as attempt_id."

    # Verify the required nested keys inside metadata
    assert "attempt_id" in proof_and_context["metadata"], \
        "Metadata lacks 'attempt_id' – needed to name the output file."
    assert "problem_id" in proof_and_context["metadata"], \
        "Metadata lacks 'problem_id' – needed for the directory structure."
    assert "proof_solver" in proof_and_context["metadata"], \
        "Metadata lacks 'proof_solver' – you need to know which solver produced this."

    # Unpack everything we need
    proof_string   = proof_and_context["proof"]
    statement      = proof_and_context["formal_statement"]
    project_dir    = proof_and_context["project_dir"]

    metadata       = proof_and_context["metadata"]
    attempt_id     = metadata["attempt_id"]
    problem_id     = metadata["problem_id"]
    solver_name    = metadata["proof_solver"]

    if max_workers is None:
        max_workers = max(os.cpu_count() - 1, 1)

    # Where the successful proof will be written.
    save_dir = os.path.join(
        "corrected_proofs", problem_id, solver_name
    )
    os.makedirs(save_dir, exist_ok=True)

    # Build every candidate proof.
    proof_variants = get_proof_variants(proof_string)

    # Each variant becomes a tiny Lean file: statement + proof.
    candidates = [
        f"{statement}\n{variant}" for variant in proof_variants
    ]

    def check_single_variant(idx, code):
        """Check a single proof variant and return (success, variant_index)"""
        # Use a temporary file with a unique name
        with tempfile.NamedTemporaryFile(mode='w', suffix='.lean', dir=project_dir, delete=False) as f:
            f.write(code)
            temp_path = f.name

        try:
            # Run Lean via lake.
            desired = 100_000
            command = [
                "lake", "env", "lean",
                f"-DmaxRecDepth={desired}",
                os.path.basename(temp_path)  # Use basename since we're in project_dir
            ]
            result = subprocess.run(
                command,
                cwd=project_dir,
                capture_output=True,
                text=True,
                timeout=30,
            )

            # Success = returncode 0 and no "error:" in stdout.
            if result.returncode == 0 and "error:" not in result.stdout:
                return (True, idx)
            
            if log_errors and "error:" in result.stdout:
                with _log_lock:
                    with open(LOG_PATH, "a", encoding="utf-8") as g:
                        g.writelines(result.stdout)
            
            return (False, idx)
            
        except Exception as e:
            print(f"Exception ({type(e).__name__}) for variant {idx}: {e}")
            proc = locals().get("result")
            if proc is not None:
                print("---- subprocess stdout ----")
                print(proc.stdout or "<no stdout>")
                print("---- subprocess stderr ----")
                print(proc.stderr or "<no stderr>")
            return (False, idx)
        finally:
            # Clean up the temp file
            try:
                os.remove(temp_path)
            except Exception:
                pass

    # Use ThreadPoolExecutor for parallel execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all variants for parallel execution
        future_to_index = {
            executor.submit(check_single_variant, idx, code): idx 
            for idx, code in enumerate(candidates)
        }

        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_index):
            success, variant_idx = future.result()
            if success:
                # Cancel remaining tasks since we found a working proof
                for f in future_to_index:
                    f.cancel()
                
                # Save the successful proof
                out_path = os.path.join(save_dir, f"{attempt_id}.txt")
                with open(out_path, "w", encoding="utf-8") as out_f:
                    out_f.write(proof_variants[variant_idx])
                
                return True

    # No variant succeeded
    return False

def check_proofs_in_parallel(proof_contexts: list[dict], parallel_workers: int = None) -> bool:
    """
    Runs multiple proof checks in parallel, numbered 1..len(proof_contexts).
    Returns True on first success.
    """
    if not proof_contexts:
        return False

    if parallel_workers is None:
        parallel_workers = 4

    with concurrent.futures.ProcessPoolExecutor(max_workers=parallel_workers) as executor:
        futures = {
            executor.submit(check_lean_proof, ctx): idx
            for idx, ctx in enumerate(proof_contexts, start=1)
        }
        for future in concurrent.futures.as_completed(futures):
            try:
                if future.result():
                    return True  # early exit on first successful attempt
            except Exception:
                pass  # optionally log
    return False


In [6]:
# # 1. Define the proof and context with the Mathlib header
# correct_proof_dict = {
#     'formal_statement': 'import Mathlib.Tactic\ntheorem two_plus_two_is_four : 2 + 2 = 4',
#     'proof': ':= by rfl',
#     'project_dir': lean_project_path,
#     'metadata': {'proof_solver': 'example_solver', 'problem_id': 'example_id', 'attempt_id': '1'}
# }


# check_lean_proof(correct_proof_dict)


# Assumptions

In [7]:
# assert that the folder structure is as expected
import os
from typing import Tuple

def check_structure(path: str):
    def check_numeric_children_consecutive(path: str, require_start_zero: bool = True) -> Tuple[int, int]:
        """
        Validate that:
        * `path` exists and is a directory with at least one child.
        * Immediate children are directories named in canonical natural-number form:
            "0", "1", "2", ... (no leading zeros except "0").
        * Their integer values form a consecutive range from min to max.
        * If require_start_zero is True, the range must start at 0.

        Returns:
            (min_value, max_value)

        Raises:
            AssertionError on any violation.
        """
        if not os.path.isdir(path):
            raise AssertionError(f"{path!r} is not a directory")
        entries = os.listdir(path)
        assert entries, f"{path!r} is empty"

        nat_canonical = re.compile(r"0|[1-9][0-9]*\Z")
        nums = []
        for name in entries:
            full = os.path.join(path, name)
            assert os.path.isdir(full), f"{full!r} is not a directory"
            assert nat_canonical.fullmatch(name), (
                f"{name!r} is not a canonical natural number ('0', '1', '2', ... without leading zeros)"
            )
            nums.append(int(name))

        min_n, max_n = min(nums), max(nums)
        if require_start_zero:
            assert min_n == 0, f"Sequence must start at 0 but starts at {min_n}"
        expected = set(range(min_n, max_n + 1))
        actual = set(nums)
        if actual != expected:
            missing = sorted(expected - actual)
            extra = sorted(actual - expected)
            msg = f"Immediate numeric directory names {sorted(entries)} do not form a consecutive range {min_n}..{max_n}"
            if missing:
                msg += f"; missing {missing}"
            if extra:
                msg += f"; unexpected {extra}"
            raise AssertionError(msg)
        return min_n, max_n
    # assert that it's a directory
    assert os.path.isdir(path), f"{path} is not a directory"

    # assert directory is not empty
    assert any(os.listdir(path)), f"{path} is empty"

    # assert only folders in the first level
    assert all(os.path.isdir(os.path.join(path, subdir)) for subdir in os.listdir(path)), f"Not all items in {path} are directories"

    # assert that all first level subdirectories are nats and ordered
    check_numeric_children_consecutive(path)

    # go into folder titled path/0 and find the set of folder names
    zero_folder = os.path.join(path, "0")
    assert os.path.isdir(zero_folder), f"{zero_folder} is not a directory"

    folder_names = {name for name in os.listdir(zero_folder) if os.path.isdir(os.path.join(zero_folder, name))}

    # assert that set of folder_names is the same across all number directories
    for subdir in os.listdir(path):
        subdir_path = os.path.join(path, subdir)
        if os.path.isdir(subdir_path) and subdir != "0":
            subdir_folder_names = {name for name in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, name))}
            assert subdir_folder_names == folder_names, f"Folder names in {subdir_path} do not match those in {zero_folder}"

    # assert that all subfolders have the same file names e.g. path/0/subfolder/[1..8].txt matches path/1/
    file_set = None
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for subdir in os.listdir(folder_path):
            assert os.path.isdir(os.path.join(folder_path, subdir)), f"{subdir} is not a directory"
            file_names = {name for name in os.listdir(os.path.join(folder_path, subdir)) if os.path.isfile(os.path.join(folder_path, subdir, name))}
            if file_set is None:
                file_set = file_names
            else:
                assert file_set == file_names, f"File names in {subdir} are not consistent with other subdirectories"

# Assumes you have a folder called proofs in the current working directory with the structure enforced by the check_structure function
check_structure("proofs_miniF2F-test")


In [8]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from datasets import load_dataset
miniF2F_test_df = load_dataset("HaimingW/miniF2F-lean4", split="test").to_pandas()

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os
import json
from typing import Any, Union

def _read_text_file(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            return f.read()

def build_structure(path: str) -> Union[dict[str, Any], str, None]:
    """
    Recursively walk `path`:
      - If it's a directory, return a dict of its children.
      - If it's a .txt file, return its text content.
      - Other files return None (and are skipped by the caller).
    """
    if os.path.isdir(path):
        out: dict[str, Any] = {}
        for entry in sorted(os.listdir(path)):
            full = os.path.join(path, entry)
            if os.path.isdir(full):
                out[entry] = build_structure(full)
            elif os.path.isfile(full) and entry.lower().endswith(".txt"):
                key = os.path.splitext(entry)[0]
                out[key] = _read_text_file(full)
        return out
    elif os.path.isfile(path) and path.lower().endswith(".txt"):
        return _read_text_file(path)
    else:
        return None  # non-txt file or missing; caller can ignore

# structure = build_structure("proofs_miniF2F-test")
# structure['0']['AI-MO_Kimina-Prover-Preview-Distill-7B']

In [11]:
# miniF2F_test_df.head(5)

In [12]:
import os
import json
from typing import Dict
from tqdm.auto import tqdm  # or: from tqdm import tqdm

In [27]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional, Callable

def check_proofs_in_parallel_output_arr(
    proof_contexts: List[dict],
    parallel_workers: int | None = None,
    on_progress: Optional[Callable[[int], None]] = None,
    log_errors = False
) -> List[int]:
    """
    Runs multiple proof checks in parallel (threaded).
    Returns a list of 1/0 aligned with the input order.
    Calls on_progress(1) each time an attempt completes.
    """
    # assert isinstance(parallel_workers, int) and parallel_workers > 1, "parallel_workers must be > 1"
    if not proof_contexts:
        return []

    results: List[int] = [0] * len(proof_contexts)

    def _run(idx: int, ctx: dict) -> Tuple[int, int]:
        name = f"proof_{idx}"
        ok = check_lean_proof(ctx, log_errors)
        return idx, 1 if ok else 0

    with ThreadPoolExecutor(max_workers=parallel_workers) as executor:
        futures = {executor.submit(_run, i, ctx): i for i, ctx in enumerate(proof_contexts, start=1)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                i, val = future.result()
            except Exception as exc:
                print(exc)
                i, val = idx, 0
            results[i - 1] = val
            if on_progress:
                on_progress(1)

    return results

In [14]:
# row1 = miniF2F_test_df.loc[8]

# correct_proof_dict = [{
#     'formal_statement': row1['header']+"\n"+row1['formal_statement']+"\n",
#     'proof': structure['8']['AI-MO_Kimina-Prover-Preview-Distill-7B'][str(i)],
#     'project_dir': lean_project_path,
#     'metadata': {'proof_solver': 'AI-MO_Kimina-Prover-Preview-Distill-7B', 'problem_id': '8', 'attempt_id': str(i)}
# } for i in range(1,25)]

# check_proofs_in_parallel_output_arr(correct_proof_dict)

In [15]:
# row1 = miniF2F_test_df.loc[0]

# correct_proof_dict = [{
#     'formal_statement': row1['header']+"\n"+row1['formal_statement']+"\n",
#     'proof': structure['0']['deepseek-ai_DeepSeek-Prover-V2-7B'][str(i)],
#     'project_dir': lean_project_path,
#     'metadata': {'proof_solver': 'deepseek-ai_DeepSeek-Prover-V2-7B', 'problem_id': '0', 'attempt_id': str(i)}
# } for i in range(1,25)]

# check_proofs_in_parallel_output_arr(correct_proof_dict)

In [16]:
# !rm -rf corrected_proofs
# !mkdir corrected_proofs

# Evaluation Loop

In [ ]:
START_IDX = 0

In [ ]:
import os
import numpy as np
from tqdm.auto import tqdm  # progress bar
from bitarray import bitarray
import json

structure_json = build_structure("proofs_miniF2F-test")

# print(structure_json)

# Derive problem/model/attempt axes from the verified folder structure
problem_ids = sorted(structure_json.keys(), key=lambda x: int(x))
problem_ids = problem_ids[START_IDX:]
first_problem = problem_ids[0]
model_names = sorted(structure_json[first_problem].keys())
# assume consistent attempt ids across models/problems (enforced by your check_structure)
attempt_ids = sorted(structure_json[first_problem][model_names[0]].keys(), key=lambda x: int(x))

num_problems = len(problem_ids)
num_models = len(model_names)
num_attempts = len(attempt_ids)

df = miniF2F_test_df

prev_success = None # will hold the result of the *last* pair

with tqdm(total=num_models * num_problems,
          desc="Evaluating (problem, model) pairs",
          unit="pair",
          leave=True,
          dynamic_ncols=True) as pbar:

    # CHANGED: Outer loop is now problems, inner loop is models
    for p_axis_idx, p_str in enumerate(problem_ids):
        p_axis_idx += START_IDX
        for m_idx, model in enumerate(model_names):
            # Show the current model & problem and the result of the previous pair
            cur_desc = f"model={model} prob={p_str}"
            if prev_success is not None:
                cur_desc += f"  prev_success={prev_success}"

            # `refresh=False` prevents an extra refresh
            pbar.set_description(cur_desc, refresh=False)

            attempts_dict = structure_json[p_str][model]
            formal_statement = miniF2F_test_df.iloc[p_axis_idx]["formal_statement"]
            header = df.iloc[p_axis_idx]["header"]

            proof_contexts = []
            for a_str in attempt_ids:
                proof_text = attempts_dict.get(a_str, "")
                proof_contexts.append({
                    "formal_statement": header+"\n"+formal_statement+"\n",
                    "proof":            proof_text,
                    "project_dir":      lean_project_path,
                    "metadata": {
                        "proof_solver": model,
                        "problem_id":   p_str,
                        "attempt_id":   a_str
                    }
                })

            # Run the checks in parallel
            outcomes = check_proofs_in_parallel_output_arr(
                proof_contexts
            )

            # Did *any* attempt succeed for this (model, problem) pair?
            this_success = any(outcomes)
            prev_success = "✔" if this_success else "✘" # store for the *next* iteration

            # Finally advance the bar by one pair
            pbar.update(1)

model=deepseek-ai_DeepSeek-Prover-V2-7B prob=161  prev_success=✔:   1%|          | 3/249 [01:26<1:58:49, 28.98s/pair]

Exception (TimeoutExpired) for variant 2: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpu97uishe.lean']' timed out after 30 seconds


model=Goedel-LM_Goedel-Prover-SFT prob=162  prev_success=✘:   2%|▏         | 5/249 [02:22<1:56:00, 28.53s/pair]           

Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpxltjq6m2.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmp8osx8e9y.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 3: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpe_7_619j.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 0: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpzpqjtfun.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 3: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpo6uf7mlg.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmp9kykhiav.lean']' timed out after 30 seconds


model=deepseek-ai_DeepSeek-Prover-V2-7B prob=162  prev_success=✔:   2%|▏         | 6/249 [02:53<1:58:43, 29.31s/pair]

...

model=Goedel-LM_Goedel-Prover-SFT prob=190  prev_success=✘:  36%|███▌      | 89/249 [46:25<1:36:17, 36.11s/pair]           

Exception (TimeoutExpired) for variant 3: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpmjnbfn2n.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 4: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpqtjt12go.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpsrpdl0pd.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 2: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmp7j20org1.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 2: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpsr4qtwfz.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 3: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmp5__g341t.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 4: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmp0qgmyrzf.lean']

model=deepseek-ai_DeepSeek-Prover-V2-7B prob=190  prev_success=✘:  36%|███▌      | 90/249 [47:01<1:35:34, 36.07s/pair]

Exception (TimeoutExpired) for variant 4: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmplx6c6d5n.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 0: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmph6u7igo7.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpyy8xw33m.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 0: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpvdw4si8e.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 1: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpe23auaqv.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 2: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmpysccij7x.lean']' timed out after 30 seconds
Exception (TimeoutExpired) for variant 2: Command '['lake', 'env', 'lean', '-DmaxRecDepth=100000', 'tmphziv8p6o.lean']

model=AI-MO_Kimina-Prover-Preview-Distill-7B prob=191  prev_success=✘:  37%|███▋      | 91/249 [47:37<1:34:34, 35.91s/pair]

In [19]:
import json
from typing import List

def results_to_nested_dict(
    results,                      # numpy array, shape: (num_problems, num_models, num_attempts)
    model_names: List[str],
    problem_ids: List[str],
) -> Dict[str, Dict[str, list]]:
    """Return {model_id: {problem_id: [0/1, ...]}}."""
    num_problems, num_models, num_attempts = results.shape
    assert len(model_names) == num_models
    assert len(problem_ids) == num_problems

    payload: Dict[str, Dict[str, list]] = {}
    for m_idx, model in enumerate(model_names):
        per_model: Dict[str, list] = {}
        for p_idx, prob in enumerate(problem_ids):
            per_model[str(prob)] = results[p_idx, m_idx, :].astype(int).tolist()
        payload[str(model)] = per_model
    return payload

def save_results_nested_json(
    results,
    model_names: List[str],
    problem_ids: List[str],
    out_path: str = "proof_outcomes_by_model.json",
) -> str:
    payload = results_to_nested_dict(results, model_names, problem_ids)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)
    return out_path

# Example:
# save_results_nested_json(results, model_names, problem_ids, "proof_outcomes_by_model.json")


In [20]:
save_results_nested_json(results, model_names, problem_ids, "proof_outcomes_by_model.json")

NameError: name 'results' is not defined

In [ ]:
def results_to_problem_nested_dict(
    results,                      # numpy array, shape: (num_problems, num_models, num_attempts)
    model_names: List[str],
    problem_ids: List[str],
) -> Dict[str, Dict[str, list]]:
    """
    Return a dict nested by problem, then model:

    {
      "<problem_id>": {
        "<model_id>": [0/1, 0/1, ...],
        ...
      },
      ...
    }
    """
    num_problems, num_models, _ = results.shape
    assert len(model_names) == num_models
    assert len(problem_ids) == num_problems

    payload: Dict[str, Dict[str, list]] = {}
    for p_idx, prob in enumerate(problem_ids):
        per_problem: Dict[str, list] = {}
        for m_idx, model in enumerate(model_names):
            per_problem[str(model)] = results[p_idx, m_idx, :].astype(int).tolist()
        payload[str(prob)] = per_problem
    return payload


def save_results_nested_by_problem_json(
    results,
    model_names: List[str],
    problem_ids: List[str],
    out_path: str = "proof_outcomes_by_problem.json",
) -> str:
    """
    Writes JSON nested by problem → model → outcomes.
    """
    payload = results_to_problem_nested_dict(results, model_names, problem_ids)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)
    return out_path


In [ ]:
save_results_nested_by_problem_json(results, model_names, problem_ids, "proof_outcomes_by_problem.json")


In [ ]:
!zip -r corrected_proofs.zip corrected_proofs